In [ ]:
# This is an unofficial open source project to interact with code interpreter
#! pip3 install "codeinterpreterapi[all]"

In [20]:
# Get OPENAI_API_KEY from user input
from getpass import getpass
OPENAI_API_KEY = getpass("Enter your API key: ")


In [21]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ['VERBOSE'] = "True"

In [22]:
import pandas as pd
from codeinterpreterapi import CodeInterpreterSession, File

In [27]:
edgar = pd.read_csv('../WasteMAP/data/edgar_per_capita.csv')

In [7]:
edgar.head(1)

,country iso3,units,1970,1971,1972,1973,1974,1975,1976,1977,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,FJI,kg/person/year,5.279816,5.935242,6.040557,6.153784,6.270426,6.381699,6.437081,6.465947,...,10.904444,11.194902,11.523695,11.846363,12.093653,12.409241,12.742104,13.05775,13.34505,13.590605


In [28]:
edgar_india = edgar[edgar['country iso3'] == 'IND']

In [9]:
edgar_india['source']='edgar'

/var/folders/r5/k5kgh1h937s5kzvhq1gry60r0000gp/T/ipykernel_86531/3719171011.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edgar_india['source']='edgar'


In [10]:
unfccc = pd.read_csv('../WasteMAP/data/unfccc_per_capita.csv')

In [11]:
unfccc_india = unfccc[unfccc['country iso3'] == 'IND']

In [12]:
unfccc_india['source']='unfccc'

/var/folders/r5/k5kgh1h937s5kzvhq1gry60r0000gp/T/ipykernel_86531/2412539651.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unfccc_india['source']='unfccc'


In [13]:
unfccc_india.shape

(1, 35)

In [14]:
india = pd.concat([edgar_india, unfccc_india],axis=0)

In [30]:
india

,country iso3,units,1970,1971,1972,1973,1974,1975,1976,1977,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,source
98,IND,kg/person/year,0.120965,0.141868,0.144716,0.147591,0.150485,0.153415,0.156691,0.160016,...,0.352222,0.363323,0.375368,0.387198,0.398341,0.409557,0.420184,0.43139,0.442945,edgar
100,IND,kg/person/year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.563260,NaN,NaN,NaN,NaN,NaN,unfccc


In [15]:
india.to_csv('india.csv')

In [16]:
india

,country iso3,units,1970,1971,1972,1973,1974,1975,1976,1977,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,source
98,IND,kg/person/year,0.120965,0.141868,0.144716,0.147591,0.150485,0.153415,0.156691,0.160016,...,0.352222,0.363323,0.375368,0.387198,0.398341,0.409557,0.420184,0.43139,0.442945,edgar
100,IND,kg/person/year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.563260,NaN,NaN,NaN,NaN,NaN,unfccc


In [32]:
unfccc_india.dropna(axis=1, how='all')

,country iso3,units,1994,2000,2010,2016,source
100,IND,kg/person/year,0.615702,0.460716,0.535977,0.56326,unfccc


In [35]:
cities = pd.read_csv('../WasteMAP/data/cities_for_map.csv')

In [40]:
india_cities = cities[cities['Country'] == 'India']

In [44]:
india_cities.columns.to_list()

['City',
 'Country',
 'Country ISO3',
 'Latitude',
 'Longitude',
 'Population',
 'Input Data Source',
 'Year of Data Collection',
 'Waste Generation Rate (tons/year)',
 'Waste Generation Rate per Capita (kg/person/day)',
 'Waste Components: Food (%)',
 'Waste Components: Green (%)',
 'Waste Components: Wood (%)',
 'Waste Components: Paper and Cardboard (%)',
 'Waste Components: Textiles (%)',
 'Waste Components: Plastic (%)',
 'Waste Components: Metal (%)',
 'Waste Components: Glass (%)',
 'Waste Components: Rubber/Leather (%)',
 'Waste Components: Other (%)',
 'Diversons: Compost (%)',
 'Diversons: Anaerobic Digestion (%)',
 'Diversons: Incineration (%)',
 'Diversons: Recycling (%)',
 'Emissions Source',
 'Year Emissions',
 'Emissions: Food (tons CH4/year)',
 'Emissions: Green (tons CH4/year)',
 'Emissions: Wood (tons CH4/year)',
 'Emissions: Paper and Cardboard (tons CH4/year)',
 'Emissions: Textiles (tons CH4/year)',
 'Emissions: Total (tons CH4/year)',
 'Emissions per Capita (kg CH

In [43]:
india_cities.to_csv('india_cities.csv')

In [45]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv('india_cities.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Columns for waste components
waste_component_cols = [
    'Waste Components: Food (%)',
    'Waste Components: Green (%)',
    'Waste Components: Wood (%)',
    'Waste Components: Paper and Cardboard (%)',
    'Waste Components: Textiles (%)',
    'Waste Components: Plastic (%)',
    'Waste Components: Metal (%)',
    'Waste Components: Glass (%)',
    'Waste Components: Rubber/Leather (%)',
    'Waste Components: Other (%)'
]

# Create the map
fig_map = px.scatter_geo(df,
                         lat='Latitude',
                         lon='Longitude',
                         hover_name='City',
                         hover_data=['Input Data Source'],
                         size='Emissions per Capita (kg CH4/person/year)',
                         color='Emissions per Capita (kg CH4/person/year)',
                         projection='orthographic',
                         title='Emissions per Capita by City in India')

# Create the layout of the app
app.layout = html.Div([
    html.H1("India Cities: Emissions and Waste Components"),
    
    # Map
    dcc.Graph(id='map', figure=fig_map),
    
    # Pie Chart
    dcc.Graph(id='pie-chart')
])

# Callback to update pie chart based on selected city in map
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('map', 'clickData')]
)
def update_pie_chart(clickData):
    if clickData is None:
        return {}
    
    # Get clicked city
    clicked_city = clickData['points'][0]['hovertext']
    
    # Filter data for clicked city
    city_data = df[df['City'] == clicked_city]
    
    # Aggregate waste components
    waste_data = city_data[waste_component_cols].mean()
    
    # Create pie chart
    fig_pie = px.pie(waste_data.reset_index(), values=waste_data, names='index', title=f'Waste Components in {clicked_city}')
    return fig_pie

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


/var/folders/r5/k5kgh1h937s5kzvhq1gry60r0000gp/T/ipykernel_86531/4177088672.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/r5/k5kgh1h937s5kzvhq1gry60r0000gp/T/ipykernel_86531/4177088672.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [46]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv('india_cities.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Columns for waste components
waste_component_cols = [
    'Waste Components: Food (%)',
    'Waste Components: Green (%)',
    'Waste Components: Wood (%)',
    'Waste Components: Paper and Cardboard (%)',
    'Waste Components: Textiles (%)',
    'Waste Components: Plastic (%)',
    'Waste Components: Metal (%)',
    'Waste Components: Glass (%)',
    'Waste Components: Rubber/Leather (%)',
    'Waste Components: Other (%)'
]

# Create the map with appropriate basemap and zoom level
fig_map = px.scatter_geo(df,
                         lat='Latitude',
                         lon='Longitude',
                         hover_name='City',
                         hover_data=['Input Data Source'],
                         size='Emissions per Capita (kg CH4/person/year)',
                         color='Emissions per Capita (kg CH4/person/year)',
                         projection='mercator',
                         title='Emissions per Capita by City in India',
                         scope='asia',
                         center={'lat': 20.5937, 'lon': 78.9629},
                         height=600)

# Create the layout of the app
app.layout = html.Div([
    html.H1("India Cities: Emissions and Waste Components"),
    
    # Map
    dcc.Graph(id='map', figure=fig_map),
    
    # Pie Chart
    dcc.Graph(id='pie-chart')
])

# Callback to update pie chart based on selected city in map
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('map', 'clickData')]
)
def update_pie_chart(clickData):
    if clickData is None:
        return {}
    
    # Get clicked city
    clicked_city = clickData['points'][0]['hovertext']
    
    # Filter data for clicked city
    city_data = df[df['City'] == clicked_city]
    
    # Aggregate waste components
    waste_data = city_data[waste_component_cols].mean()
    
    # Create pie chart
    fig_pie = px.pie(waste_data.reset_index(), values=waste_data, names='index', title=f'Waste Components in {clicked_city}')
    return fig_pie

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [50]:
cities

,City,Country,Country ISO3,Latitude,Longitude,Population,Input Data Source,Year of Data Collection,Waste Generation Rate (tons/year),Waste Generation Rate per Capita (kg/person/day),...,Non Compostable Not Targeted Total (%),Precipitation Zone,k: Food,k: Green,k: Wood,k: Paper and Cardboard,k: Textiles,MEF: Compost,Fraction of Waste to Methane Capture Landfills (%),Methane Capture Efficiency (%)
5002,Kota,India,IND,25.173702,75.857419,1001694.0,World Bank,2016,216504.0,0.592159,...,0.0,Moderately Dry,0.18,0.09,0.018,0.036,0.036,0.004244,0.0,60.0
5003,Kota,India,IND,25.173702,75.857419,1001694.0,World Bank,2016,216504.0,0.592159,...,0.0,Moderately Dry,0.18,0.09,0.018,0.036,0.036,0.004244,0.0,60.0
5004,Kota,India,IND,25.173702,75.857419,1001694.0,World Bank,2016,216504.0,0.592159,...,0.0,Moderately Dry,0.18,0.09,0.018,0.036,0.036,0.004244,0.0,60.0
5005,Kota,India,IND,25.173702,75.857419,1001694.0,World Bank,2016,216504.0,0.592159,...,0.0,Moderately Dry,0.18,0.09,0.018,0.036,0.036,0.004244,0.0,60.0
5006,Kota,India,IND,25.173702,75.857419,1001694.0,World Bank,2016,216504.0,0.592159,...,0.0,Moderately Dry,0.18,0.09,0.018,0.036,0.036,0.004244,0.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14509,Thiruvananthapuram,India,IND,8.488227,76.947551,969298.0,UN Habitat,2021,154395.0,0.436398,...,0.1,Wet,0.34,0.15,0.030,0.060,0.060,0.004244,0.0,60.0
14510,Thiruvananthapuram,India,IND,8.488227,76.947551,969298.0,UN Habitat,2021,154395.0,0.436398,...,0.1,Wet,0.34,0.15,0.030,0.060,0.060,0.004244,0.0,60.0
14511,Thiruvananthapuram,India,IND,8.488227,76.947551,969298.0,UN Habitat,2021,154395.0,0.436398,...,0.1,Wet,0.34,0.15,0.030,0.060,0.060,0.004244,0.0,60.0
14512,Thiruvananthapuram,India,IND,8.488227,76.947551,969298.0,UN Habitat,2021,154395.0,0.436398,...,0.1,Wet,0.34,0.15,0.030,0.060,0.060,0.004244,0.0,60.0


In [51]:
len(cities['City'].unique())

46

In [55]:
cities_with_kota = [
    "Kota", "Kozhikode", "Jaipur", "Tirunelveli", "Amritsar", "Allahabad",
    "Greater Hyderabad", "Kochi", "Leh", "Pune", "Patna",
    "Jodhpur", "Nagpur", "Shimla", "Rudrapur", "Bengaluru",
    "Pimpri-Chinchwad", "Kanpur", "Rajkot", "Tenali", "Lucknow",
    "Ludhiana", "Bhopal", "Faridabad", "Cuttack", "Ahmedabad",
    "Vishakhapatnam", "Nashik", "Dhanbad", "Delhi", "Srinagar",
    "Ghaziabad", "Imphal", "Surat", "Vijaywada", "Chennai",
    "Indore", "Coimbatore", "Greater Mumbai", "Guwahati", "Gwalior",
    "Bhubaneswar", "Mysore", "Warangal", "Mangalore", "Thiruvananthapuram"
]

In [56]:
len(cities_with_kota)

46